In [1]:
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
!pip install flask_sqlalchemy

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
!pip install sqlalchemy psycopg2 mysql-connector-python pymysql

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [4]:
!pip install pyodbc

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [1]:
import mysql.connector
import pandas as pd

# MySQL connection details
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
}

# Establish MySQL connection
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Step 1: Create the Data Warehouse Database (if not exists)
cursor.execute("CREATE DATABASE IF NOT EXISTS gravity_books_DWH;")
cursor.execute("USE gravity_books_DWH;")  # Switch to the new database

# Step 2: Create Star Schema Tables
star_schema_queries = [
    """
    CREATE TABLE IF NOT EXISTS DimOrder (
        method_id_SK INT AUTO_INCREMENT PRIMARY KEY,
        method_id_bk INT,
        method_name VARCHAR(100),
        source_system_code VARCHAR(50),
        start_date DATE,
        end_date DATE,
        is_current BOOLEAN
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS DimBook (
        book_id_SK INT AUTO_INCREMENT PRIMARY KEY,
        book_id_BK INT,
        language_id_BK INT,
        author_id_BK INT,
        publisher_id_BK INT,
        isbn13 VARCHAR(13),
        publication_date DATE,
        num_pages INT,
        language_name VARCHAR(50),
        publisher_name VARCHAR(200),
        author_name VARCHAR(200),
        source_system_code VARCHAR(50),
        start_date DATE,
        end_date DATE,
        is_current BOOLEAN
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS DimCustomer (
        customer_id_SK INT AUTO_INCREMENT PRIMARY KEY,
        customer_id_BK INT,
        address_id_BK INT,
        country_id_BK INT,
        status_id_BK INT,
        first_name VARCHAR(100),
        last_name VARCHAR(100),
        email VARCHAR(200),
        country_name VARCHAR(100),
        street_name VARCHAR(200),
        city VARCHAR(100),
        address_status VARCHAR(50),
        source_system_code VARCHAR(50),
        start_date DATE,
        end_date DATE,
        is_current BOOLEAN
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS DimDate (
        date_Id_SK INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        month INT,
        day INT,
        start_date DATE,
        end_date DATE,
        is_current BOOLEAN
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS FactOrder (
        order_id_SK INT AUTO_INCREMENT PRIMARY KEY,
        line_id INT,
        order_history_id INT,
        book_id INT,
        customer_id INT,
        shipping_method_id INT,
        order_status_id INT,
        order_status VARCHAR(50),
        price DECIMAL(10,2),
        shipping_cost DECIMAL(10,2),
        source_system_code VARCHAR(50),
        created_at DATETIME,
        FOREIGN KEY (book_id) REFERENCES DimBook(book_id_SK),
        FOREIGN KEY (customer_id) REFERENCES DimCustomer(customer_id_SK),
        FOREIGN KEY (shipping_method_id) REFERENCES DimOrder(method_id_SK)
    )
    """
]

# Execute table creation queries separately
for query in star_schema_queries:
    cursor.execute(query)

print("Star Schema tables created successfully.")

# Close connection
cursor.close()
conn.close()
print("Database setup completed.")


Star Schema tables created successfully.
Database setup completed.


In [2]:
import mysql.connector

# MySQL connection details
source_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books"  # Source database
}

target_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books_DWH"  # Target data warehouse
}

# Establish MySQL connections
source_conn = mysql.connector.connect(**source_db_config)
source_cursor = source_conn.cursor()

target_conn = mysql.connector.connect(**target_db_config)
target_cursor = target_conn.cursor()

# Function to check if a table exists in the target database
def check_table_exists(cursor, table_name):
    cursor.execute("SHOW TABLES LIKE %s", (table_name,))
    return cursor.fetchone() is not None

# Extract, Transform, and Load (ETL) process
def load_dim_tables():
    try:
        # Check if DimOrder table exists in DWH
        if not check_table_exists(target_cursor, "DimOrder"):
            raise Exception("Error: Table 'DimOrder' does not exist in 'gravity_books_DWH'.")

        # Load DimOrder
        source_cursor.execute("SELECT method_id, method_name FROM gravity_books.shipping_method")
        orders = source_cursor.fetchall()
        for order in orders:
            target_cursor.execute("""
                INSERT INTO DimOrder (method_id_bk, method_name, source_system_code, is_current)
                VALUES (%s, %s, 'gravity_books', TRUE)
            """, order)

        # Load DimBook
        source_cursor.execute("""
            SELECT b.book_id, b.isbn13, b.publication_date, b.num_pages, l.language_name, 
                   p.publisher_name, a.author_name 
            FROM gravity_books.book b 
            JOIN gravity_books.book_language l ON b.language_id = l.language_id
            JOIN gravity_books.publisher p ON b.publisher_id = p.publisher_id
            JOIN gravity_books.book_author ba ON b.book_id = ba.book_id
            JOIN gravity_books.author a ON ba.author_id = a.author_id
        """)
        books = source_cursor.fetchall()
        for book in books:
            target_cursor.execute("""
                INSERT INTO DimBook (book_id_BK, isbn13, publication_date, num_pages, language_name, publisher_name, author_name, source_system_code, is_current)
                VALUES (%s, %s, %s, %s, %s, %s, %s, 'gravity_books', TRUE)
            """, book)

        # Load DimCustomer
        source_cursor.execute("""
            SELECT c.customer_id, c.first_name, c.last_name, c.email, co.country_name, a.street_name, a.city, s.address_status
            FROM gravity_books.customer c
            JOIN gravity_books.customer_address ca ON c.customer_id = ca.customer_id
            JOIN gravity_books.address a ON ca.address_id = a.address_id
            JOIN gravity_books.country co ON a.country_id = co.country_id
            JOIN gravity_books.address_status s ON ca.status_id = s.status_id
        """)
        customers = source_cursor.fetchall()
        for customer in customers:
            target_cursor.execute("""
                INSERT INTO DimCustomer (customer_id_BK, first_name, last_name, email, country_name, street_name, city, address_status, source_system_code, is_current)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, 'gravity_books', TRUE)
            """, customer)

        # Load FactOrder
        source_cursor.execute("""
            SELECT ol.line_id, oh.history_id, ol.book_id, c.customer_id, co.shipping_method_id, os.status_id, os.status_value, ol.price, sm.cost, co.order_date
            FROM gravity_books.order_line ol
            JOIN gravity_books.order_history oh ON ol.order_id = oh.order_id
            JOIN gravity_books.cust_order co ON ol.order_id = co.order_id
            JOIN gravity_books.customer c ON co.customer_id = c.customer_id
            JOIN gravity_books.shipping_method sm ON co.shipping_method_id = sm.method_id
            JOIN gravity_books.order_status os ON oh.status_id = os.status_id
        """)
        orders = source_cursor.fetchall()
        for order in orders:
            target_cursor.execute("""
                INSERT INTO FactOrder (line_id, order_history_id, book_id, customer_id, shipping_method_id, order_status_id, order_status, price, shipping_cost, source_system_code, created_at)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, 'gravity_books', %s)
            """, order)

        target_conn.commit()
        print("Data successfully loaded into star schema.")

    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")
        target_conn.rollback()
    except Exception as e:
        print(f"Custom Error: {e}")
        target_conn.rollback()

# Run ETL process
load_dim_tables()

# Close connections
source_cursor.close()
source_conn.close()
target_cursor.close()
target_conn.close()
print("ETL process completed and MySQL connections closed.")


Data successfully loaded into star schema.
ETL process completed and MySQL connections closed.


In [3]:
import mysql.connector

# MySQL connection details
source_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books"  # Source database
}

target_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books_DWH"  # Target data warehouse
}

# Establish MySQL connection to the target database
conn = mysql.connector.connect(**target_db_config)
cursor = conn.cursor()

# Function to check if a table exists in the source database
def check_table_exists(cursor, table_name):
    query = f"SHOW TABLES LIKE '{table_name}'"
    cursor.execute(query)
    result = cursor.fetchone()
    return result is not None

# Define SQL update queries with corrected table names
update_queries = [
    # Update language_id_BK using source book table
    """
    UPDATE DimBook d
    JOIN gravity_books.book b ON d.book_id_BK = b.book_id
    SET d.language_id_BK = b.language_id
    WHERE d.language_id_BK IS NULL;
    """,
    
    # Update author_id_BK using source book_author table
    """
    UPDATE DimBook d
    JOIN gravity_books.book_author ba ON d.book_id_BK = ba.book_id
    SET d.author_id_BK = ba.author_id
    WHERE d.author_id_BK IS NULL;
    """,
    
    # Update publisher_id_BK using source book table
    """
    UPDATE DimBook d
    JOIN gravity_books.book b ON d.book_id_BK = b.book_id
    SET d.publisher_id_BK = b.publisher_id
    WHERE d.publisher_id_BK IS NULL;
    """,
    
    # ✅ Update language_name using source language table
    """
    UPDATE DimBook d
    JOIN gravity_books.language l ON d.language_id_BK = l.language_id
    SET d.language_name = l.language_name
    WHERE d.language_name IS NULL;
    """,
    
    # Update publisher_name using source publisher table
    """
    UPDATE DimBook d
    JOIN gravity_books.publisher p ON d.publisher_id_BK = p.publisher_id
    SET d.publisher_name = p.publisher_name
    WHERE d.publisher_name IS NULL;
    """,
    
    # Update author_name using source author table
    """
    UPDATE DimBook d
    JOIN gravity_books.author a ON d.author_id_BK = a.author_id
    SET d.author_name = a.author_name
    WHERE d.author_name IS NULL;
    """,
    
    # Update start_date with publication_date from source book table
    """
    UPDATE DimBook d
    JOIN gravity_books.book b ON d.book_id_BK = b.book_id
    SET d.start_date = b.publication_date
    WHERE d.start_date IS NULL;
    """,
    
    # Ensure end_date is NULL for current records
    """
    UPDATE DimBook
    SET end_date = NULL
    WHERE is_current = 1;
    """
]

# Execute update queries
try:
    conn.start_transaction()

    # Check if the 'language' table exists before executing the query
    if check_table_exists(cursor, 'language'):
        cursor.execute(update_queries[3])  # The query related to 'language_name'
        print("Successfully updated language_name in DimBook.")
    
    # Execute the other queries, excluding the language-related one if necessary
    for i, query in enumerate(update_queries):
        if i != 3:  # Skip the 'language' table update if it doesn't exist
            cursor.execute(query)
    
    conn.commit()
    print("Successfully updated NULL values in DimBook.")
except mysql.connector.Error as err:
    conn.rollback()
    print(f"MySQL Error: {err}")
finally:
    cursor.close()
    conn.close()


Successfully updated NULL values in DimBook.


In [4]:
import mysql.connector

# MySQL connection details
source_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books"  # Source database
}

target_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books_DWH"  # Target data warehouse
}

# Establish MySQL connections
source_conn = mysql.connector.connect(**source_db_config)
source_cursor = source_conn.cursor()

target_conn = mysql.connector.connect(**target_db_config)
target_cursor = target_conn.cursor()

# Function to check if a table exists in the target database
def check_table_exists(cursor, table_name):
    cursor.execute("SHOW TABLES LIKE %s", (table_name,))
    return cursor.fetchone() is not None

# Extract, Transform, and Load (ETL) for DimCustomer
def load_dim_customer():
    try:
        # Check if DimCustomer table exists
        if not check_table_exists(target_cursor, "DimCustomer"):
            raise Exception("Error: Table 'DimCustomer' does not exist in 'gravity_books_DWH'.")

        # Extract data from source tables using JOIN
        source_cursor.execute("""
            SELECT 
                c.customer_id, 
                c.first_name, 
                c.last_name, 
                c.email, 
                a.address_id AS address_id_BK, 
                co.country_id AS country_id_BK, 
                s.status_id AS status_id_BK, 
                co.country_name, 
                a.street_name, 
                a.city, 
                s.address_status
            FROM gravity_books.customer c
            JOIN gravity_books.customer_address ca ON c.customer_id = ca.customer_id
            JOIN gravity_books.address a ON ca.address_id = a.address_id
            JOIN gravity_books.country co ON a.country_id = co.country_id
            JOIN gravity_books.address_status s ON ca.status_id = s.status_id
        """)

        customers = source_cursor.fetchall()

        # Load data into DimCustomer
        for customer in customers:
            target_cursor.execute("""
                INSERT INTO DimCustomer (
                    customer_id_BK, first_name, last_name, email, 
                    address_id_BK, country_id_BK, status_id_BK, 
                    country_name, street_name, city, address_status, 
                    source_system_code, is_current
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 'gravity_books', TRUE)
            """, customer)

        target_conn.commit()
        print("✅ Data successfully loaded into DimCustomer.")

    except mysql.connector.Error as err:
        print(f"❌ MySQL Error: {err}")
        target_conn.rollback()
    except Exception as e:
        print(f"⚠ Custom Error: {e}")
        target_conn.rollback()

# Run ETL process
load_dim_customer()

# Close connections
source_cursor.close()
source_conn.close()
target_cursor.close()
target_conn.close()
print("✅ ETL process completed and MySQL connections closed.")

✅ Data successfully loaded into DimCustomer.
✅ ETL process completed and MySQL connections closed.


In [5]:
import mysql.connector

# MySQL connection details
source_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books"  # Source database
}

target_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books_DWH"  # Target data warehouse
}

# Establish MySQL connection to the target database
conn = mysql.connector.connect(**target_db_config)
cursor = conn.cursor()

# Function to check if a table exists in the source database
def check_table_exists(cursor, table_name):
    query = f"SHOW TABLES LIKE '{table_name}'"
    cursor.execute(query)
    result = cursor.fetchone()
    return result is not None

# Define SQL update queries with corrected JOINs
update_queries = [
    # ✅ Fixing address_id_BK update: Proper JOIN with customer_address (alias ca)
    """
    UPDATE DimCustomer dc
    JOIN gravity_books.customer c ON dc.customer_id_BK = c.customer_id
    JOIN gravity_books.customer_address ca ON c.customer_id = ca.customer_id  -- Added missing JOIN
    SET dc.address_id_BK = ca.address_id
    WHERE dc.address_id_BK IS NULL;
    """,

]

# Execute update queries
try:
    conn.start_transaction()

    # Execute all update queries
    for query in update_queries:
        cursor.execute(query)

    conn.commit()
    print("✅ Successfully updated NULL values in DimBook and DimCustomer.")
except mysql.connector.Error as err:
    conn.rollback()
    print(f"❌ MySQL Error: {err}")
finally:
    cursor.close()
    conn.close()
    print("✅ MySQL connection closed.")


✅ Successfully updated NULL values in DimBook and DimCustomer.
✅ MySQL connection closed.


In [6]:
import mysql.connector

# MySQL connection details
source_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books"  # Source database
}

target_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books_DWH"  # Target data warehouse
}

# Establish MySQL connection to the target database
conn = mysql.connector.connect(**target_db_config)
cursor = conn.cursor()

# Function to check if a table exists in the source database
def check_table_exists(cursor, table_name):
    query = f"SHOW TABLES LIKE '{table_name}'"
    cursor.execute(query)
    result = cursor.fetchone()
    return result is not None

# Define SQL update queries with corrected JOINs
update_queries = [
    # ✅ Fixed country_id_BK update query
    """
    UPDATE DimCustomer dc
    JOIN gravity_books.address a ON dc.address_id_BK = a.address_id
    JOIN gravity_books.country co ON a.country_id = co.country_id
    SET dc.country_id_BK = co.country_id
    WHERE dc.country_id_BK IS NULL;
    """
]

# Execute update queries
try:
    conn.start_transaction()

    # Execute all update queries
    for query in update_queries:
        cursor.execute(query)

    conn.commit()
    print("✅ Successfully updated NULL values in DimCustomer.")
except mysql.connector.Error as err:
    conn.rollback()
    print(f"❌ MySQL Error: {err}")
finally:
    cursor.close()
    conn.close()
    print("✅ MySQL connection closed.")


✅ Successfully updated NULL values in DimCustomer.
✅ MySQL connection closed.


In [7]:
import mysql.connector

# MySQL connection details
target_db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "gravity_books_DWH"  # Target data warehouse
}

# Establish MySQL connection to the target database
conn = mysql.connector.connect(**target_db_config)
cursor = conn.cursor()

# Define the corrected SQL update query
update_query = """
    UPDATE DimCustomer dc
    JOIN gravity_books.customer_address ca 
        ON dc.customer_id_BK = ca.customer_id
    JOIN gravity_books.address_status s 
        ON ca.status_id = s.status_id
    SET dc.status_id_BK = s.status_id
    WHERE dc.status_id_BK IS NULL;
"""

# Execute the update query
try:
    conn.start_transaction()
    cursor.execute(update_query)
    conn.commit()
    print("✅ Successfully updated NULL values in DimCustomer.")
except mysql.connector.Error as err:
    conn.rollback()
    print(f"❌ MySQL Error: {err}")
finally:
    cursor.close()
    conn.close()
    print("✅ MySQL connection closed.")

✅ Successfully updated NULL values in DimCustomer.
✅ MySQL connection closed.
